In [1]:
import pormake as pm
import numpy as np
import csv
from collections import defaultdict

db = pm.Database()
loc = pm.Locator()

Fe_oct_a = pm.BuildingBlock(bb_file="pi-d_building_blocks/old_bb/Fe_oct.xyz")
Fe_oct_b = Fe_oct_a.make_chiral_building_block()
Fe_oct = Fe_oct_a
DHBQ = pm.BuildingBlock(bb_file="pi-d_building_blocks/old_bb/DHBQ.xyz")

pm.log.disable_print()
pm.log.disable_file_print()


2023-07-24 09:52:45.125408: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-24 09:52:45.176533: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-24 09:52:45.177512: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-24 09:52:45.910429: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
>>> There are atoms without bond: Fe_oct, [(4, 'Xx'), (5, 'Xx'), (6, 'Xx')].
>>> There are atoms without bond: DHBQ, [(14, 'Xx'), (15, 'Xx')].
>>> Console logs (under WARNING level) are disabled.


In [2]:
def expand_topo(topo):
    x = 1
    y = 1
    z = 1
    edges = topo.edge_indices
    neigh = topo.neighbor_list
    for i in edges:

        ## below set lists unique neighbors for the edges. If 1, this means expansion is required.
        if len(set([j.index for j in neigh[i]])) == 1:
            dist = [abs(d) for d in neigh[i][0].distance_vector]

            ## expand topology along the "major" edge direction to allow for alternating node sequence.
            if x == 1 and dist.index(max(dist)) == 0:
                x = 2
            if y == 1 and dist.index(max(dist)) == 1:
                y = 2
            if z == 1 and dist.index(max(dist)) == 2:
                z = 2

    return topo * (x, y, z)

In [3]:
## prep topologies for patterned node assignment

topo_codes = []
file = open('cn34_pid_topology_list.csv')
csvreader = csv.reader(file)

for row in csvreader:
    for topo in row:
        topo_codes.append(topo)

topos = []

for topo in topo_codes:
    cur_topo = db.get_topo(topo)
    cur_topo = expand_topo(cur_topo)

    if cur_topo.n_nodes % 2 == 1:
        ## expand in one of the directions to ensure even number of nodes
        cur_topo = cur_topo * (1, 1, 2)
        
    topos.append(cur_topo)

In [6]:
## check rmsd and topology scale here (check # nodes again since expansion took place)

node_limit = 80
cn3_topos = []

for topo in topos:
    
    if topo.n_nodes > node_limit:
        continue
    
    if max(topo.unique_cn) == 3:
        cn3_topos.append(topo)

print(len(cn3_topos))

63


In [7]:

## attempt generation!
test_builder = pm.Builder(planarity_enforcement=True, angle_threshold=30, check_tetrahedral=False)
successful_cases = []

for topo in cn3_topos:
    
    print(topo.name)
    
    for chiral_grouping in [1, 2, 3, 4, 6, 8, 12, 16]:
        for chiral_begin_flip in [True, False]:

            current_node = {}
            current_edge = {}    

            ## initialize nodes with Fe_oct first (no chirality dependence... is this good?)
            for i, node in enumerate(topo.unique_cn):
                current_node[i] = Fe_oct

            ## initialize edges
            for i, edge in enumerate(topo.unique_edge_types):
                current_edge[tuple(edge)] = DHBQ

            bbs = test_builder.make_bbs_by_type(topo, current_node, current_edge)
            calc_permutation = test_builder.extract_permutation(topo, current_node, current_edge)
            calc_permutation = {i : perm for i, perm in enumerate(calc_permutation)}

            ## consider chirality of Fe_oct nodes

            chiral_counter = 0
            chiral_check_count = chiral_grouping + 1
            chiral_flip = chiral_begin_flip

            for node in topo.node_indices:

                if chiral_counter % chiral_check_count == 0:
                    chiral_flip = not chiral_flip
                    chiral_counter += 1

                if chiral_flip:
                    bbs[node] = Fe_oct_a
                else:
                    bbs[node] = Fe_oct_b

                chiral_counter += 1


            try:
                mof = test_builder.build(topo, bbs, calc_permutation)
                mof.view()
                successful_cases.append(topo.name)

            except:
                continue



bcu-f
bcu-h
bpa
bpb
bpc
bpe
bpg
bpi
bpl
bpm
bpp
bps
bpt
bpv
bto
clh
ctn-d
eta
eta-c
etb
etc
etd
ete
etf
etg
eth
etj
etn
hha
lig
nnd
nod
noh
noi
noj
nos
nta
ntb
nva
nvb
ofo
oft
pbp
pbz
pcu-g
pcu-h
phl
pyo


/home/chong/miniconda3/envs/pormake/lib/python3.8/site-packages/pormake-0.0.2-py3.8.egg/pormake/builder.py:667: RuntimeWarning: invalid value encountered in scalar divide
  ratio = rmsd / slot_min_rmsd[key]
/home/chong/miniconda3/envs/pormake/lib/python3.8/site-packages/pormake-0.0.2-py3.8.egg/pormake/builder.py:677: RuntimeWarning: invalid value encountered in scalar divide
  ratio = rmsd / slot_min_rmsd[key]
/home/chong/miniconda3/envs/pormake/lib/python3.8/site-packages/pormake-0.0.2-py3.8.egg/pormake/builder.py:667: RuntimeWarning: divide by zero encountered in scalar divide
  ratio = rmsd / slot_min_rmsd[key]
/home/chong/miniconda3/envs/pormake/lib/python3.8/site-packages/pormake-0.0.2-py3.8.egg/pormake/builder.py:677: RuntimeWarning: divide by zero encountered in scalar divide
  ratio = rmsd / slot_min_rmsd[key]


srd-l
srs
sxc-d
ths
tta
ttg
twt
uni-d
utg
utj
utm
utp
wix
wiz
zme


In [8]:
successful_cases

['bcu-h',
 'bpc',
 'bpc',
 'etb',
 'etc',
 'etc',
 'lig',
 'pbz',
 'pcu-h',
 'pcu-h',
 'srs',
 'srs',
 'srs',
 'twt',
 'twt',
 'wiz']